# Backtesting MutantSupertrend

In [1]:
# from __future__ import (absolute_import, division, print_function,
#                         unicode_literals)
import datetime
# import os.path
import sys

import pandas as pd
import numpy as np
import backtrader as bt
import backtrader.indicators as ta

import matplotlib
import tkinter
matplotlib.use('TKAgg')
# matplotlib.use('QT5Agg')

from mutant.model import MutantSupertrend
from mutant.strategy import MutantSupertrendBacktrader

raw_data_path = "../data/BTCUSD_latest.csv"
# raw_data_path = "../data/Raw_BTCUSDT1708-2303.csv"

## Load data

In [2]:
dataframe = pd.read_csv(raw_data_path,
                                parse_dates=True,
                                index_col=0)
dataframe.index = pd.to_datetime(dataframe.index, format='ISO8601')

In [3]:
dataframe

,open,high,low,close,volume
date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,7564.90
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.00
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,1117.54
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,51.17
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,599.99
...,...,...,...,...,...
2023-07-04 21:15:00,30794.96,30794.96,30790.20,30790.21,202250.26
2023-07-04 21:16:00,30790.21,30791.08,30790.20,30791.08,210922.27
2023-07-04 21:17:00,30791.08,30791.08,30780.94,30780.95,407579.78


In [4]:
start = "2019-09-09"
end = "2022-08-31"
df = dataframe[start:end]

In [5]:
load_params = np.load("out.npz", allow_pickle=True)
load_params.keys()

KeysView(NpzFile 'out.npz' with keys: populations, objective, control_params, info, op_objective...)

In [6]:
load_params['control_params']

array([[None, None, None, ...,
        {'ema_length': array([40]), 'st_factor_main': array([29]), 'st_atr_main': array([22]), 'st_factor_tf2': array([30]), 'st_atr_tf2': array([52]), 'tsv_length': array([13]), 'tsv_ma_length': array([8]), 'adx_length': array([5]), 'adx_threshold': array([44])},
        {'ema_length': array([283]), 'st_factor_main': array([21]), 'st_atr_main': array([20]), 'st_factor_tf2': array([29]), 'st_atr_tf2': array([26]), 'tsv_length': array([20]), 'tsv_ma_length': array([14]), 'adx_length': array([4]), 'adx_threshold': array([50])},
        None],
       [{'ema_length': array([131]), 'st_factor_main': array([11]), 'st_atr_main': array([26]), 'st_factor_tf2': array([25]), 'st_atr_tf2': array([29]), 'tsv_length': array([16]), 'tsv_ma_length': array([7]), 'adx_length': array([20]), 'adx_threshold': array([68])},
        {'ema_length': array([221]), 'st_factor_main': array([23]), 'st_atr_main': array([17]), 'st_factor_tf2': array([50]), 'st_atr_tf2': array([21]), 't

In [7]:
load_params = np.load("out-op_control_param.npz", allow_pickle=True)
control_params = {}
for key in load_params:
    control_params[key] = load_params[key]

control_params

{'ema_length': array([129]),
 'st_factor_main': array([15]),
 'st_atr_main': array([10]),
 'st_factor_tf2': array([53]),
 'st_atr_tf2': array([58]),
 'tsv_length': array([28]),
 'tsv_ma_length': array([11]),
 'adx_length': array([7]),
 'adx_threshold': array([27])}

In [8]:
# control_params = {
#     'ema_length': np.array([116]),
#     'st_factor_main': np.array([14]),
#     'st_atr_main': np.array([26]),
#     'st_factor_tf2': np.array([30]),
#     'st_atr_tf2': np.array([31]),
#     'tsv_length': np.array([20]),
#     'tsv_ma_length': np.array([15]),
#     'adx_length': np.array([3]),
#     'adx_threshold': np.array([67])
# }

In [9]:
model = MutantSupertrend()
print(model.params)
model.update_params(control_params)
print(model.params)

{'ema_length': array([200]), 'st_factor_main': array([16]), 'st_atr_main': array([6]), 'st_factor_tf2': array([3]), 'st_atr_tf2': array([10]), 'tsv_length': array([13]), 'tsv_ma_length': array([7]), 'adx_length': array([3]), 'adx_threshold': array([60])}
{'ema_length': array([129]), 'st_factor_main': array([15]), 'st_atr_main': array([10]), 'st_factor_tf2': array([53]), 'st_atr_tf2': array([58]), 'tsv_length': array([28]), 'tsv_ma_length': array([11]), 'adx_length': array([7]), 'adx_threshold': array([27])}


In [10]:
trade_reports = []
sharp_reports = []
drawdown_reports = []
total_sessions = 1
for i in range(total_sessions):
    # backtest_length = 1440
    # start = np.random.choice(len(dataframe) - backtest_length)
    # end = start + backtest_length
    # df = dataframe.iloc[start:end]
    df = dataframe[start:end]
    print(df)
    df = df.groupby(pd.Grouper(freq='5Min')).agg({"open": "first", 
                                                  "high": "max",
                                                  "low": "min",
                                                  "close": "last",
                                                  "volume": "sum"})
    df = df.dropna()
    data = bt.feeds.PandasData(dataname=df, datetime=None,)
    cerebro = bt.Cerebro()
    cerebro.addstrategy(MutantSupertrendBacktrader, model, print_log=True)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='mutant_trade')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='mutant_drawdown')
    cerebro.addanalyzer(
        bt.analyzers.SharpeRatio,
        timeframe=bt.TimeFrame.Days, 
        compression=1, 
        factor=365,
        annualize =True,
        _name='mutant_sharpe'
    )
    cerebro.addsizer(bt.sizers.PercentSizer, percents=99)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0004)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    result = results[0]
    trade_reports.append(result.analyzers.mutant_trade.get_analysis())
    sharp_reports.append(result.analyzers.mutant_sharpe.get_analysis())
    drawdown_reports.append(result.analyzers.mutant_drawdown.get_analysis())
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

                         open      high       low     close      volume
date                                                                   
2019-09-09 00:00:00  10381.24  10392.05  10377.79  10379.77    83061.37
2019-09-09 00:01:00  10380.78  10392.41  10376.45  10392.41    67564.73
2019-09-09 00:02:00  10390.20  10391.85  10382.89  10383.87    62115.10
2019-09-09 00:03:00  10386.03  10392.37  10382.89  10390.14    47645.62
2019-09-09 00:04:00  10390.00  10391.21  10386.68  10390.08    29230.04
...                       ...       ...       ...       ...         ...
2022-08-31 23:55:00  20039.99  20049.77  20035.03  20047.55  1685646.07
2022-08-31 23:56:00  20047.55  20052.07  20039.49  20043.22  2168835.24
2022-08-31 23:57:00  20043.22  20056.67  20041.86  20046.30  2065079.77
2022-08-31 23:58:00  20046.30  20052.58  20040.00  20051.51  1377181.95
2022-08-31 23:59:00  20051.51  20057.78  20042.42  20050.02  1949965.19

[1564212 rows x 5 columns]
Starting Portfolio Value: 100000.00


In [11]:
sharp_reports

[OrderedDict([('sharperatio', 1.062919658674874)])]

In [12]:
""" Trade report is provided by Backtrader.analyzers.TradeAnalyzer

"""
def get_win_rate(trade_raport):
    """ Get the win rate from trade report
    
    Win rate = number of win trades / number of total trades
    """
    total_trades = trade_raport['total']['total']
    win_trades = trade_raport['won']['total']
    win_rate = win_trades / total_trades
    return win_rate

def get_roi(trade_raport, init_protfolio_value=100000.0):
    """ Get the ROI from trade report

    ROI = PNL / Initial Portfolio Value
    """
    pnl = trade_raport['pnl']['net']['total']
    roi = pnl / init_protfolio_value
    return roi

def get_drawdown(drawdown_report):
    """ Get the drawdown from drawdown report

    """
    drawdown = drawdown_report['drawdown']
    return drawdown

In [13]:
sessions_win = 0
sessions_win_rate = 0
roi_s = []
for i in range(total_sessions):
    trade_report = trade_reports[i]
    if trade_report['total']['total'] > 0:
        drawdown_report = drawdown_reports[i]
        win_rate = get_win_rate(trade_report)
        roi = get_roi(trade_report)
        roi_s.append(roi)
        if roi > 0:
            sessions_win += 1
        drawdown = get_drawdown(drawdown_report)
        print("total_trades: {}, win_rate: {:.2f},  roi: {:.2f}%, drawdown: {:.2f}%.".format(trade_report['total']['total'], win_rate, roi*100, -drawdown))
    else:
        print("No trade.")
        
# sessions_win_rate = sessions_win/total_sessions * 100
# roi_avg = sum(roi_s) / len(roi_s)
# print("sessions_win_rate: {:.2f}%, roi_avg: {:.4f}%.".format(sessions_win_rate, roi_avg*100)) 

total_trades: 227, win_rate: 0.38,  roi: 261.16%, drawdown: -32.42%.
